## Updating the ACL groups for ReferenceData records in DEV

This notebook is intended to provide a short practical example on how to use the patching funtionality available on the OSDU storage API in order to update the ACL groups for the reference data in the DEV environment. This was performed in order to make all reference data searchable and therefore referencable on the Seimic Data Bank metadata work when moving the records from custom to well-know schemas.

In [1]:
from libs.osdu_service.osdu_http_client import OsduHttpClient
import json

# Adding .env file variables as environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
"""
Let's use the OSDU CLI client for the task in DEV.
"""

osdu_env = "npequinor-dev"
# osdu_env = "npequinor-test"
# osdu_env = "equinor-data"
osdu_env = OsduHttpClient(osdu_env, client_type="public-client")

In [3]:
"""
Testing wether our client has access to on of the records we aim to target.
It should have access to both the SEARCH and STORAGE API's to perform its tasks.
"""

# data = {
#         "kind": "osdu:wks:reference-data--SeismicTraceDataDimensionalityType:1.0.0",
#         "query": "id: \"npequinor-dev:reference-data--SeismicTraceDataDimensionalityType:3D\""
#     }

# resp_search = osdu_env.app_post_returning_json(
#     service_relative_uri="search/v2/query",
#     payload=data
#     )

# resp_storage = osdu_env.app_get_returning_json(
#     "storage/v2/records/npequinor-dev:reference-data--SeismicTraceDataDimensionalityType:3D"
#     )

# assert resp_search[0].get("id") == resp_storage["id"]

In [67]:
"""
Let's search for all the objects' ids we aim to modify
"""

payload =     {
    "kind": "osdu:wks:reference-data*:*",
    "returnedFields": ["id","kind", "acl"],
    "limit": 1000
    }

service_url = f"search/v2/query_with_cursor"

response = osdu_env.app_query_with_cursor(
    service_url,
    payload
)

print(f"Records found: {len(response)}")

Records found: 16973


In [68]:
""" 
Let's check which records have the correct acl groups.
"""

# records_ok_acl = [i["id"] for i in resp if i.get("acl").get("viewers")[0] == "data.default.viewers@npequinor-dev.dataservices.energy"]
# records_non_ok_acl = [i["id"] for i in resp if i.get("acl").get("viewers")[0] == "data.osdudevops.viewers@npequinor-dev.dataservices.energy"]

acl_group1 = "data.default.viewers@npequinor-dev.dataservices.energy"
acl_group2 = "data.osdudevops.viewers@npequinor-dev.dataservices.energy"

records_acl1 = []
records_acl2 = []
records_both_acl = []

for i in response:
    viewers = i.get("acl").get("viewers")
    if (acl_group1 in viewers) and (acl_group2 in viewers):
        records_both_acl.append(i)
    elif acl_group1 in viewers:
        records_acl2.append(i)
    elif acl_group2 in viewers:
        records_acl1.append(i)

print(
    f"""
    Total records: {len(response)}
    Records with both acl = {len(records_both_acl)},
    Records with acl1: {acl_group1} = {len(records_acl1)},
    Records with acl2: {acl_group2} = {len(records_acl2)}
    """
)

assert len(records_acl1) + len(records_acl2) + len(records_both_acl) == len(response)       


    Total records: 16973
    Records with both acl = 2,
    Records with acl1: data.default.viewers@npequinor-dev.dataservices.energy = 11814,
    Records with acl2: data.osdudevops.viewers@npequinor-dev.dataservices.energy = 5157
    


In [71]:
records_both_acl[0]

{'kind': 'osdu:wks:reference-data--SeismicTraceDataDimensionalityType:1.0.0',
 'acl': {'viewers': ['data.default.viewers@npequinor-dev.dataservices.energy',
   'data.osdudevops.viewers@npequinor-dev.dataservices.energy',
   'testacl@npequinor-dev.dataservices.energy'],
  'owners': ['data.osdudevops.owners@npequinor-dev.dataservices.energy']},
 'id': 'npequinor-dev:reference-data--SeismicTraceDataDimensionalityType:3D'}

In [72]:
"""
Let's create our UPDATE/PATCH loop, as we do not want to use PUT method.
This way we preserve the currect version of the records while save some storage.
"""
import requests

records_patched = []
records_patch_failed = []

for record in response[:]:
    try:
        
        patch_body = { 
                    "query": { 
                        "ids": [
                        record["id"]
                        ]
                    }, 
                    "ops": [
                        { 
                        "op": "replace", 
                        "path": "/acl/viewers", 
                        "value": [
                            "data.default.viewers@npequinor-dev.dataservices.energy",
                            "data.osdudevops.viewers@npequinor-dev.dataservices.energy",
                        ]
                        }
                    ] 
                    }

        osdu_base_url = osdu_env.env_variables["url"]
        patch_url = f"{osdu_base_url}/storage/v2"
        headers = osdu_env.osdu_client.get_headers()

        response = requests.patch(
            f"{patch_url}/records",
            data=json.dumps(patch_body),
            headers=headers
        )
        
        records_patched.append(record["id"])
        
        response
    
    except:
        
        records_patch_failed.append(record["id"])
        

In [76]:
print(
    f"""
    # Records attempted to patch: {len(response)}
    Records succesfully patched: {len(records_patched)},
    Records failed when patching: {len(records_patch_failed)}
    """
)


    # Records attempted to patch: 16973
    Records succesfully patched: 16973,
    Records failed when patching: 0
    
